# Set environment

In [ ]:
# # http://pytorch.org/
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [ ]:
# !git clone https://github.com/gilbertolem/ProgGen

In [ ]:
# from os import chdir, getcwd
# chdir("ProgGen")
# from sys import path
# path.append(getcwd())

# Train

In [ ]:
import torch
import utils.data_tools as data_tools
from utils.nets import ProgGen_RNN
from pickle import load, dump
import matplotlib.pyplot as plt

xml_directory = "XML_Tunes/"
torch.manual_seed(999)
use_gpu = torch.cuda.is_available()

In [ ]:
# Load vocabulary
words_text2num = load(open("maps/words_text2num.txt",'rb'))
vocab_size = len(words_text2num)

# Create training data
filters = {'author':'Charlie Parker', 'style':None}
X_train, X_val = data_tools.musicxml2tensor(xml_directory, words_text2num, filters = filters) # (Seq x Batch x vocab_size)
train_data = data_tools.TuneData(X_train)
val_data = data_tools.TuneData(X_val)

In [ ]:
# Construct Neural Net
embed_size = 100
hidden_size = 256
num_layers = 1
dropout = 0
bidirectional = False
rnn_type = 'lstm'
model = ProgGen_RNN(rnn_type, vocab_size, embed_size, hidden_size, num_layers, dropout, bidirectional)
loss_fn = torch.nn.CrossEntropyLoss()

# Define loader
sampler = torch.utils.data.RandomSampler(train_data)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 372, sampler = sampler, num_workers = 1 if use_gpu else 4)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = 372, num_workers = 1 if use_gpu else 4)

if use_gpu:
    model = model.cuda()
    loss_fn = loss_fn.cuda()

In [ ]:
# Define loss function and optimizer
lr = 1e-2
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from utils.training import train
epochs = 500
losses = train(epochs, model, optim, train_loader, val_loader, loss_fn, use_gpu)

In [ ]:
plt.semilogy(losses[0], label='Train')
plt.semilogy(losses[1], label='Val')
plt.legend()
plt.show()
import numpy as np
print(losses[0][-1])
print(losses[1][-1])
idx = np.argmin(losses[1])
print(losses[0][idx])
print(losses[1][idx])

# Generate something

In [ ]:
from utils.generating import generate_progression

model_name = "model"
initial_chord = "4C_maj"
tune_len = 32
top = 10

prog = generate_progression(initial_chord, tune_len, top, model_name, verbose = False)
print("Generated Progression:\n")
print(prog)